In [ ]:
#libruaris import#
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import urllib
from bs4 import BeautifulSoup
import datetime
import ast
import csv
import requests
from IPython.display import HTML
import time
import random
from itertools import zip_longest

%matplotlib inline

In [ ]:
#the main purpose of this function is to get the information of each job in every single search page#
##input:page url##
##output:Couple Lists of information such as title,locations,companies#
def get_page_info(url):
    #request url for getting the html content#
    response = requests.get(url)
    content = response.content
    #use beautiful soup to parse the content#
    html_soup = BeautifulSoup(content,'lxml')
  
    job_urls =[]
    #find h2 tag title class in soup#
    for divTag in html_soup.find_all('h2',{'class':'title'}):
        #find a tag in those h2 tag title class#
        for aTag in divTag.find_all('a'):
            #obtain url for each job#
            url ='https://ca.indeed.com'+ aTag.get('href')
            #obtain job titles#
            job_titles.append(aTag.get('title'))
            #save them in a list#
            job_urls.append(url)

    #find div tag sjcl class in soup#
    for divTag in html_soup.find_all('div',{'class':'sjcl'}):
        for spanTag in divTag.find_all('span',{'class':'company'}):#find span tag in those div tag#
          if spanTag != None:
            #obtain company name and save them in a list#
            companies.append(spanTag.get_text().replace("\n",""))
          else:
            companies.append(None)

        #find span tag location class in those div tag#
        #obtain locations#
        #save it in a list#
        try:
          ratings = divTag.find('span',{'class':'ratingsContent'}).get_text()
          company_ratings.append(ratings.replace("\n",""))
        except AttributeError:
          ratings = float('nan')
          company_ratings.append(ratings)

        #find span tag date class in those div div tag#
        try:
          #obtain the post date#
          #save it in a list#
          loc = divTag.find('span',{'class':'location accessible-contrast-color-location'}).get_text()
          locations.append(loc.replace("\n",""))
        except AttributeError:
          loc = float('nan')
          locations.append(loc)
     #find span tag date class in those div div tag#
    for divTag in html_soup.find_all('div',{'class':'result-link-bar'}):
        #obtain the post date#
        for spanTag in divTag.find_all('span',{'class':'date'}):
          #save it in a list#     
          if spanTag != None:
            post_dates.append(spanTag.get_text().replace("\n",""))
          else:
            post_dates.append(None)  

    #print(post_dates)
    time.sleep(0.5)
    return job_urls

In [ ]:
# dictionary with skills used to parse job descriptions, categories become columns in the resulting DataFrame
skills_dict = {
        # tech skills
        'Excel': ['Excel'],
        'Python': ['Python'],
        'R': ['R ', ' R ', 'R,', 'R/'], # 'R' surrounded by spaces and signs
        'Java': ['Java', 'JVM'],
        'Scala': ['Scala'],
        'C/C++': ['C/C++', 'C++', ' C '],   # 'C' surrounded by spaces and signs
        'MATLAB': ['MATLAB'],
        'SAS': ['SAS'],
        'SQL/databases': ['SQL', 'databases'],
        'Oracle':['Oracle'],
        'SPSS': ['SPSS'],
        'Machine Learning': ['Machine Learning', 'ML'],
        'Data Mining/Analytics': ['Data Mining', 'DM', 'Analytics'],
        'Visualisation': ['Visualisation', 'Visualization'],
        'Big Data': ['Big Data', 'Spark', 'kafka', 'Hive',
                     'beam', 'Hadoop', 'MapReduce', 'Hbase',
                     'Coudera', 'Hortonworks'],
        'AWS Cloud': ['AWS'],
        'Jason': ['Jason'],
        'xml': ['xml'],
        'Azure': ['Azure'],
        'Google Cloud': ['Google Cloud'],
        'IBM': ['IBM'],
        'Operations research': ['Operations research'],
        'DevOps': ['DevOps', 'TDD', 'test-driven'],
        'Git':['GitHub', 'Git', 'version control'],

        # soft skills
        'presentation' : ['communication', 'presentation'],
        'management' : ['management', 'Data management'],
        'agile' : ['agile'],
        'SDLC' : ['SDLC', 'sdlc', 'software development', 'lifecycle'],
        'decision making' : ['decision making', 'decision analysis'],
        'problem solving': ['problem solving'],
        'project_management': ['project management'],
        'leadership': ['leadership'],
        'consulting': ['consulting', 'consultant']
    }

In [ ]:
##input:the list of url of each specific job##
##output:required skill data frame##
def job_qual_info(job_links):
  ### Get html for each job url
  skill_info_dict = dict()
  #loop through the list and request the content of each job#
  for job_link in job_links:
    time.sleep(0.5)
    job_response = requests.get(job_link)
    job_soup = BeautifulSoup(job_response.text,'lxml')#parse the content by using soup#
    #find salary#
    sal = job_soup.find('div',{'class':'jobsearch-DesktopStickyContainer'})

    if sal != None:
      spanTag = sal.find('span',{'class':'icl-u-xs-mr--xs'})
      if spanTag != None:
        Salary.append(spanTag.get_text().replace("\n",""))
      else:
        Salary.append(None)
    else:
      Salary.append(None)
    
    skill_info_dict[job_link] = {}
    #obtain job description#
    jd = job_soup.find('div',{'class':'jobsearch-jobDescriptionText'}).get_text()

    #save 1 into the dictionary if has the specific skill#
    for skill_type, skill_names in skills_dict.items():
      skill_found = 0   # variable used to store results of the intermediate check (loop below)
      for skill_name in skill_names:        # loop over all skills in the sublist of 'skills_keywords_dict'
        if jd.find(skill_name) != -1:     # if skill from the sublist is found, set 'category_found' to 1
           skill_found = 1
      skill_info_dict[job_link][skill_type] = skill_found   # skill set to 1 if found, 0 if not, in 'results_dict'
    #print(results_dict)
    
  #convert the dictionary into a dataframe
  skills_df = pd.DataFrame(skill_info_dict).T.reset_index()
  
  return skills_df
   
    


In [ ]:
job_titles = []
companies = []
company_ratings =[]
locations = []
post_dates = []
Salary = []


################################################################################



#page_url = 'https://ca.indeed.com/jobs?q=data+scientist&l=Canada'
#response = requests.get(page_url)
#content = response.content

#html_soup = BeautifulSoup(content,'lxml')
###########################################################
### Get the max page number and iterate to get all pages url
#for tag in html_soup.find_all('head'):
    #for metaTag in tag.find_all('meta',{'name':'description'}):
        #txt = metaTag.get('content')
#temp = int(int("".join(filter(str.isdigit,txt)))/15)
###########################################################
temp_url = 'https://ca.indeed.com/jobs?q=data+scientist&l=Canada&start='


page_urls = []
for i in range(0,80,10):
    page_urls.append(temp_url +str(i))


### Go through page by page
appended_data = []
for page_url in page_urls:

  ### Get info from main page
    job_urls = get_page_info(page_url)
    ### get detail info from individual job page
    skills_df=job_qual_info(job_urls)


    appended_data.append(skills_df)
    time.sleep(0.5)

appended_data = pd.concat(appended_data)
appended_data.reset_index( drop=True,inplace=True)
del appended_data['index']


In [ ]:
df = pd.DataFrame(list(zip_longest(job_titles, companies,company_ratings,locations,post_dates,Salary)), 
               columns =['job_titles', 'companies','company_ratings','locations','post_dates','Salary']) 
df_csv=pd.concat([df,appended_data], axis=1)

In [ ]:
df_csv

,job_titles,companies,company_ratings,locations,post_dates,Salary,Excel,Python,R,Java,Scala,C/C++,MATLAB,SAS,SQL/databases,Oracle,SPSS,Machine Learning,Data Mining/Analytics,Visualisation,Big Data,AWS Cloud,Jason,xml,Azure,Google Cloud,IBM,Operations research,DevOps,Git,presentation,management,agile,SDLC,decision making,problem solving,project_management,leadership,consulting
0,Senior Data Scientist,MaxSold,3.6,NaN,8 days ago,None,1,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0
1,"Data Scientist, Solar & Storage (Remote)",Power Factors,NaN,NaN,30+ days ago,None,0,1,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0
2,Data Scientist,Charger Logistics Inc.,3.6,NaN,3 days ago,None,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Junior Data Scientist with Python experience,Samiti Technology,NaN,"Toronto, ON",Today,$250 - $350 a day,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Data Scientist,CakeAI,NaN,"Toronto, ON",Today,None,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,"Senior Data Scientist, GANs TORONTO, ONSOFTWARE",Tonal,NaN,"Toronto, ON",23 days ago,None,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
116,Lead Data Scientist,Peak Power,NaN,"Toronto, ON",8 days ago,None,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,1,0
117,Data Analyst,Microsoft,4.2,"Vancouver, BC",30+ days ago,None,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0
118,"Senior Data Scientist, KPMG Lighthouse",KPMG,4.0,"Montréal, QC",22 days ago,None,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [ ]:
from google.colab import files
df_csv.to_csv(r'indeed_csv1.csv') 
files.download('indeed_csv1.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>